Update Data functions

In [1]:
import os 
import sys
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
# Importar modulos propios en notebook
project_root = Path(os.getcwd()).parent
sys.path.append(str(project_root))

In [3]:
# from logger.logger import logger
import dependencies as dp

In [ ]:
from processes.transform.silver import bronze_df
from processes.extract.silver import dataframes as silver_df
from db.engines import engine_silver, conn_silver

In [3]:
from db.engines import engine_silver, conn_silver

2024-12-02 15:27:16,399 - INFO - Successful connection to schema bronze
2024-12-02 15:27:16,428 - INFO - Successful connection to schema silver
2024-12-02 15:27:16,468 - INFO - Successful connection to schema gold


In [5]:
from model.table_relations import related_bron_silv

In [6]:
def create_combined_key (df, key_columns, date_column=None):
    '''
    Creates a unique key from different key columns given.

    input:
        - df (DataFrame): Dataframe from which key_columns are taken.
        - key_columns (list): List of strings of the ckay column names.
        - date_column str: Name of a column of date type (optional).

    output:
        - combined_key (Series): Serie of the combined key created from key_columns and date_column.

    '''
    combined_key = df[key_columns[0]].astype(str)

    for key in key_columns[1:]:
        combined_key += '_' + df[key].astype(str)

    if date_column:
        combined_key += '_' + df[date_column].astype(str)

    return combined_key

In [7]:
def get_modified_data (df_new_data, df_ref_data, key_columns, date_column=None):
    '''
    Identifies and retrieves modified rows by comparing a DataFrame with new data 
    against a reference DataFrame representing the current state in the database.

    Parameters:
        - df_new_data (DataFrame): DataFrame containing the new data to compare.
        - df_ref_data (DataFrame): DataFrame with the reference data from the database.
        - key_columns (list of str): List of column names used to generate a unique 
          combined key for identifying rows.
        - date_column (str, optional): Name of a date column included in the combined 
          key for more precise row identification (default is None).

    Returns:
        - df_mod_data (DataFrame): A DataFrame containing only the rows with modified 
          data, where any column differs between the new and reference data. 
    '''

    # create combined_key for each df 
    df_new_data = df_new_data.assign(
        combined_key=create_combined_key(
            df=df_new_data,
            key_columns=key_columns,
            date_column=date_column)
        )

    df_ref_data = df_ref_data.assign(
        combined_key=create_combined_key(
            df=df_ref_data,
            key_columns=key_columns,
            date_column=date_column)
        )

    merge_data = df_new_data.merge(
        df_ref_data, on='combined_key',
        suffixes=('_new', '_ref')
        )

    # Make sure both DataFrames have the same column names and index
    new_merge_data = merge_data.filter(regex='_new$').sort_index(axis=1)
    df_ref_data = merge_data.filter(regex='_ref$').sort_index(axis=1)

    # Otherwise column names are adjusted
    new_merge_data.columns = new_merge_data.columns.str.replace('_new$', '', regex=True)
    df_ref_data.columns = df_ref_data.columns.str.replace('_ref$', '', regex=True)

    # Normalize data: fill None/NaN and convert to consistent types
    new_merge_data = new_merge_data.fillna("").astype(str)
    df_ref_data = df_ref_data.fillna("").astype(str)


    # Compare rows where there are differences
    df_mod_data = merge_data[(new_merge_data != df_ref_data).any(axis=1)]

    return df_mod_data

In [ ]:
from sqlalchemy import text

def update_modified_data (df_mod_data, key_columns, db_tbl_name, conn):
    '''
    Updates rows in a database table based on modified data provided in a Pandas DataFrame.

    imput:
        - df_mod_data (DataFrame): Dataframe with rows of data that has been modified.
        - key_columns (list of str): List of column names used to generate a unique 
          combined key for identifying rows. 
        - bd_tbl_name (str): String with the name of the database table name to be updated. 
        - conn (sqlalchemy.engine.base.Connection): Database base engine base connection.
    
    output:
        None
    '''
    for _, fila in df_mod_data.iterrows():
        print(f"Procesando fila {_}:")
        print(fila)

        # Dic of values fot SET CLause (without "_new")
        valores = {
            col.replace('_new', ''): fila[col]
            for col in df_mod_data.filter(regex='_new$').columns
        }

        # print("Valores para SET:", valores)

        # SET and WHERE Clauses
        set_clause = ', '.join([f'"{col}" = :{col}' for col in valores.keys()])
        where_clause = " AND ".join(
            [f'"{key}" = :key_{key}' for key in key_columns]
        )
        query = f'UPDATE "{db_tbl_name}" SET {set_clause} WHERE {where_clause}'

        # Create query parameters
        params = valores.copy()
        params.update({
            f'key_{key}': fila[f"{key}_new"] for key in key_columns
        })

        # print("Query generado:", query)
        # print("Parámetros generados:", params)

        # Execute query
        with conn as conn:
            conn.begin()
            conn.execute(text(query), params)
            conn.commit()

In [ ]:
for key in related_bron_silv.keys():
    #if key in {'master_table', 'master_table_mult', 'fact_table', 'fact_table_mult'}:
    for table in related_bron_silv[key]:
        dp.logger.info(f'Processing updating data process fot table: "{table['tbl_silv']}"')
        # Accede a los valores del diccionario interno
        tbl_bron = table['tbl_bron']
        tbl_silv = table['tbl_silv']
        key_columns = table['key_columns']
        date_column = table['date_column']
        
        # Llama a la función usando los valores extraídos
        df_mod_data = get_modified_data(
            df_new_data=bronze_df[tbl_bron], 
            df_ref_data=silver_df[tbl_silv], 
            key_columns=key_columns,
            date_column=date_column
        )
        # Llamar a la función de modificacíón si hay datos para modificar
        if not df_mod_data.empty:
            dp.logger.info(f'There is data to update in table "{tbl_silv}"')
        
            try:                
                update_modified_data (
                    df_mod_data=df_mod_data,
                    key_columns=key_columns,
                    db_tbl_name=tbl_silv,
                    conn=conn_silver)
                
                dp.logger.info(f'Update data has been insert succesfully into table "{tbl_silv}": {e}')

            except Exception as e:
                dp.logger.error(f"An error has occurred trying to insert update data into table '{tbl_silv}': {e}")
                continue
        
        else:
            dp.logger.info(f'There is not update data to insert into table "{tbl_silv}"')

Validación - Tests 

In [9]:
df_mod_data = pd.DataFrame({'Codigo_new': [13], 'Nombre_new':['MOD_DATA_TEST_3'],})
key_columns = ['Codigo']
db_tbl_name = 'Empresas'

In [10]:
from sqlalchemy import text

# Iterar sobre las filas del DataFrame
for _, fila in df_mod_data.iterrows():
    print(f"Procesando fila {_}:")
    print(fila)

    # Diccionario de valores para SET (nombres sin "_new")
    valores = {
        col.replace('_new', ''): fila[col]
        for col in df_mod_data.filter(regex='_new$').columns
    }

    print("Valores para SET:", valores)

    # Cláusulas SET y WHERE
    set_clause = ', '.join([f'"{col}" = :{col}' for col in valores.keys()])
    where_clause = " AND ".join(
        [f'"{key}" = :key_{key}' for key in key_columns]
    )
    query = f'UPDATE "{db_table_name}" SET {set_clause} WHERE {where_clause}'

    # Crear parámetros para la consulta
    params = valores.copy()
    params.update({
        f'key_{key}': fila[f"{key}_new"] for key in key_columns
    })

    print("Query generado:", query)
    print("Parámetros generados:", params)

    # Execute query
    with conn_silver as conn:
        conn.begin()
        conn.execute(text(query), params)
        conn.commit()


Procesando fila 0:
Codigo_new                 13
Nombre_new    MOD_DATA_TEST_3
Name: 0, dtype: object
Valores para SET: {'Codigo': 13, 'Nombre': 'MOD_DATA_TEST_3'}
Query generado: UPDATE "Empresas" SET "Codigo" = :Codigo, "Nombre" = :Nombre WHERE "Codigo" = :key_Codigo
Parámetros generados: {'Codigo': 13, 'Nombre': 'MOD_DATA_TEST_3', 'key_Codigo': 13}


In [13]:
type(conn_silver)

sqlalchemy.engine.base.Connection